In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
spark = SparkSession.builder \
        .appName("ETL-with-spark") \
        .getOrCreate()

In [3]:
df = spark.read.parquet('green_tripdata_2022-01.parquet')

In [4]:
for i in ['passenger_count', 'RateCodeID', 'payment_type', 'trip_type']:
    df = df.withColumn(i, col(i).cast(IntegerType()))

In [5]:
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RateCodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2022-01-01 07:14:21|  2022-01-01 07:15:33|                 N|         1|          42|          42|              1|         0.44|        3.5|  0.5|    0.

In [6]:
trip_time_in_mins = unix_timestamp(
		col('lpep_dropoff_datetime')) - unix_timestamp(col('lpep_pickup_datetime'))
df = df.withColumn('trip_time_in_mins', round(trip_time_in_mins / 60, 2))
df.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-----------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RateCodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|trip_time_in_mins|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+-----------------+
|       2| 2022-01-01 07:14:21|  2022-01-01 07:15:33|                 N|         1|          42|          42|

In [ ]:
type_trip = when(col(trip_time_in_mins) >= 60 * 6, 'Long Trip').when(col(trip_time_in_mins) < 60 * 6 and col(trip_time_in_mins) > 60 * 2, 'Medium Trip').otherwise('Short Trip')


In [ ]:
df_zone = spark.read.csv('data', inferSchema=True, header=True)
df_zone = df_zone.filter(col('Borough') != 'Unknown')
df_zone.show()

In [3]:
host = 'localhost'
port = '5432'
database = 'etl_with_spark_nytaxi'
user = 'postgres'
password = 'tanduy2407'
url = 'jdbc:postgresql://{0}:{1}/{2}'.format(host, port, database)
properties = {
    'user': user,
    'password': password,
    'driver': 'org.postgresql.Driver'}
df.write.jdbc(url=url, table='taxi_zone_lookup', mode='overwrite',
                properties=properties)

In [4]:
df.show()

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|  Green Zone|
|         3|        Bronx|Allerton/Pelham G...|  Green Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|  Green Zone|
|         6|Staten Island|Arrochar/Fort Wad...|  Green Zone|
|         7|       Queens|             Astoria|  Green Zone|
|         8|       Queens|        Astoria Park|  Green Zone|
|         9|       Queens|          Auburndale|  Green Zone|
|        10|       Queens|        Baisley Park|  Green Zone|
|        11|     Brooklyn|          Bath Beach|  Green Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City| Yellow Zone|
|        14|     Brookly